## Installation

In [1]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is not available


In [2]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.3.1 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [4]:
# import system lib
import os
import time
import shutil
import pathlib
import itertools

# import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers
#import tensorflow_addons as tfa
from keras import backend as K
# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

print ('modules loaded')

modules loaded


## Download & Unzip

In [5]:
!kaggle datasets download -d abduzzami/allenh5dill10aug4stepdownhasina

Dataset URL: https://www.kaggle.com/datasets/abduzzami/allenh5dill10aug4stepdownhasina
License(s): MIT
 67% 5.00M/7.48M [00:00<00:00, 29.3MB/s]
100% 7.48M/7.48M [00:00<00:00, 39.9MB/s]


In [6]:
import zipfile
import os

# Define the path to the ZIP file and the directory to extract to
zip_file_path = "/content/allenh5dill10aug4stepdownhasina.zip"
extract_to_path = "/content/masks/"

# Create the directory if it doesn't exist
os.makedirs(extract_to_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("Extraction complete.")



Extraction complete.


In [7]:
!kaggle datasets download -d raddar/tuberculosis-chest-xrays-montgomery

Dataset URL: https://www.kaggle.com/datasets/raddar/tuberculosis-chest-xrays-montgomery
License(s): unknown
 98% 571M/585M [00:16<00:00, 58.2MB/s]
100% 585M/585M [00:16<00:00, 37.2MB/s]


In [8]:
!kaggle datasets download -d raddar/tuberculosis-chest-xrays-shenzhen

Dataset URL: https://www.kaggle.com/datasets/raddar/tuberculosis-chest-xrays-shenzhen
License(s): unknown
100% 3.50G/3.51G [00:48<00:00, 117MB/s]
100% 3.51G/3.51G [00:48<00:00, 76.9MB/s]


In [9]:
!kaggle datasets download -d tawsifurrahman/tuberculosis-tb-chest-xray-dataset

Dataset URL: https://www.kaggle.com/datasets/tawsifurrahman/tuberculosis-tb-chest-xray-dataset
License(s): copyright-authors
 98% 653M/663M [00:06<00:00, 116MB/s]
100% 663M/663M [00:06<00:00, 114MB/s]


In [10]:
!kaggle datasets download -d nikhilpandey360/chest-xray-masks-and-labels

Dataset URL: https://www.kaggle.com/datasets/nikhilpandey360/chest-xray-masks-and-labels
License(s): CC0-1.0
100% 9.56G/9.58G [02:11<00:00, 53.9MB/s]
100% 9.58G/9.58G [02:11<00:00, 78.2MB/s]


In [11]:
import zipfile
import os

# Define the path to the ZIP file and the directory to extract to
zip_file_path = "/content/chest-xray-masks-and-labels.zip"
extract_to_path = "/content/availabel_masks/"

# Create the directory if it doesn't exist
os.makedirs(extract_to_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("Extraction complete.")

Extraction complete.


In [12]:
import zipfile
import os

# Define the path to the ZIP file and the directory to extract to
zip_file_path = "/content/tuberculosis-chest-xrays-montgomery.zip"
extract_to_path = "/content/montgomery/"

# Create the directory if it doesn't exist
os.makedirs(extract_to_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("Extraction complete.")

Extraction complete.


In [13]:
import zipfile
import os

# Define the path to the ZIP file and the directory to extract to
zip_file_path = "/content/tuberculosis-chest-xrays-shenzhen.zip"
extract_to_path = "/content/Shenzhen/"

# Create the directory if it doesn't exist
os.makedirs(extract_to_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("Extraction complete.")

Extraction complete.


In [14]:
import zipfile
import os

# Define the path to the ZIP file and the directory to extract to
zip_file_path = "/content/tuberculosis-tb-chest-xray-dataset.zip"
extract_to_path = "/content/4200images/"

# Create the directory if it doesn't exist
os.makedirs(extract_to_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("Extraction complete.")

Extraction complete.


## Segmentation

In [37]:
#Number of images
import os

def count_images_in_directory(directory_path):
    # Initialize a dictionary to store image counts per subdirectory
    image_counts = {}

    # Walk through all subdirectories and files
    for root, dirs, files in os.walk(directory_path):
        # Filter for image files (assuming .png format)
        image_files = [f for f in files if f.endswith('.png')]

        # Count the number of image files in the current directory
        num_images = len(image_files)

        # Store the count in the dictionary with the directory path as the key
        if num_images > 0:
            image_counts[root] = num_images

    return image_counts

# Define the path to the segmented images directory
segmented_images_path = '/content/Montgomery_segmented'

# Get the image counts for each subdirectory
image_counts = count_images_in_directory(segmented_images_path)

# Print the number of images in each subdirectory
for directory, count in image_counts.items():
    print(f"Number of images in {directory}: {count}")


Number of images in /content/Montgomery_segmented/Available_masks: 138
Number of images in /content/Montgomery_segmented/Montgomery_Custom_Masks/Tuberculosis: 58
Number of images in /content/Montgomery_segmented/Montgomery_Custom_Masks/Normal: 80
Number of images in /content/Montgomery_segmented/Montgomery_Available_masks/Tuberculosis: 58
Number of images in /content/Montgomery_segmented/Montgomery_Available_masks/Normal: 80
Number of images in /content/Montgomery_segmented/Custom_masks: 138


In [16]:
#montgomery
from PIL import Image
import os

# Define the paths
images_path = '/content/montgomery/images/images'
masks_path = '/content/available_masks/Lung Segmentation/masks'
output_path = '/content/Montgomery_segmented/Available_masks'

# Create the output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# List all images
images = sorted(os.listdir(images_path))

# Process each image and its corresponding mask
for img_name in images:
    # Construct mask name assuming it follows the same naming convention as images
    mask_name = img_name  # Adjust this if the naming convention is different

    mask_path = os.path.join(masks_path, mask_name)

    # Check if the corresponding mask exists
    if os.path.exists(mask_path):
        # Load image and mask
        img = Image.open(os.path.join(images_path, img_name)).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        # Apply mask to image
        segmented_img = Image.composite(img, Image.new('RGB', img.size, (0, 0, 0)), mask)

        # Save the segmented image
        segmented_img.save(os.path.join(output_path, img_name))
    else:
        print(f"Mask for {img_name} not found. Skipping this image.")

print("Segmentation complete. Segmented images saved to:", output_path)


Segmentation complete. Segmented images saved to: /content/Montgomery_segmented/Available_masks


In [18]:
#Montogomery Custom Model Segmentation
from PIL import Image
import os

# Define the paths
images_path = '/content/montgomery/images/images'
masks_path = '/content/masks/AllEnh5Dill10Aug4/Montgomery'
output_path = '/content/Montgomery_segmented/Custom_masks'

# Create the output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# List all images
images = sorted(os.listdir(images_path))

# Process each image and its corresponding mask
for img_name in images:
    # Construct mask name assuming it follows the same naming convention as images
    mask_name = img_name  # Adjust this if the naming convention is different

    mask_path = os.path.join(masks_path, mask_name)

    # Check if the corresponding mask exists
    if os.path.exists(mask_path):
        # Load image and mask
        img = Image.open(os.path.join(images_path, img_name)).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        # Resize the image and mask to (224, 224)
        img = img.resize((224, 224))
        mask = mask.resize((224, 224))

        # Apply mask to image
        segmented_img = Image.composite(img, Image.new('RGB', img.size, (0, 0, 0)), mask)

        # Save the segmented image
        segmented_img.save(os.path.join(output_path, img_name))
    else:
        print(f"Mask for {img_name} not found. Skipping this image.")

print("Segmentation complete. Segmented images saved to:", output_path)


Segmentation complete. Segmented images saved to: /content/Montgomery_segmented/Custom_masks


In [20]:
#Shenzhen available masks
import os
from PIL import Image
import numpy as np

# Define the paths
images_path = '/content/Shenzhen/images/images'
masks_path = '/content/available_masks/Lung Segmentation/masks'
output_path = '/content/Shenzhen_segmented/Available_Masks'

# Ensure the output directory exists
os.makedirs(output_path, exist_ok=True)

# Get list of all images in the images directory
image_files = [f for f in os.listdir(images_path) if f.endswith('.png')]

for image_file in image_files:
    # Construct full file paths
    image_path = os.path.join(images_path, image_file)
    mask_file = image_file.replace('.png', '_mask.png')
    mask_path = os.path.join(masks_path, mask_file)

    # Check if both image and mask exist
    if not os.path.exists(image_path):
        print(f"Image {image_file} is missing, skipping.")
        continue
    if not os.path.exists(mask_path):
        print(f"Mask for {image_file} is missing, skipping.")
        continue

    # Open image and mask
    image = Image.open(image_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")  # Convert mask to grayscale

    # Convert to numpy arrays
    image_np = np.array(image)
    mask_np = np.array(mask)

    # Create segmented image
    segmented_image_np = image_np.copy()
    segmented_image_np[mask_np == 0] = 0  # Apply mask

    # Convert back to PIL Image and save
    segmented_image = Image.fromarray(segmented_image_np)
    segmented_image.save(os.path.join(output_path, image_file))

print("Segmentation completed.")





Mask for CHNCXR_0343_1.png is missing, skipping.
Mask for CHNCXR_0037_0.png is missing, skipping.
Mask for CHNCXR_0359_1.png is missing, skipping.
Mask for CHNCXR_0189_0.png is missing, skipping.
Mask for CHNCXR_0484_1.png is missing, skipping.
Mask for CHNCXR_0565_1.png is missing, skipping.
Mask for CHNCXR_0197_0.png is missing, skipping.
Mask for CHNCXR_0351_1.png is missing, skipping.
Mask for CHNCXR_0193_0.png is missing, skipping.
Mask for CHNCXR_0345_1.png is missing, skipping.
Mask for CHNCXR_0206_0.png is missing, skipping.
Mask for CHNCXR_0482_1.png is missing, skipping.
Mask for CHNCXR_0192_0.png is missing, skipping.
Mask for CHNCXR_0493_1.png is missing, skipping.
Mask for CHNCXR_0498_1.png is missing, skipping.
Mask for CHNCXR_0219_0.png is missing, skipping.
Mask for CHNCXR_0349_1.png is missing, skipping.
Mask for CHNCXR_0216_0.png is missing, skipping.
Mask for CHNCXR_0499_1.png is missing, skipping.
Mask for CHNCXR_0201_0.png is missing, skipping.
Mask for CHNCXR_0184

In [27]:
#Upload missing_images.csv
#missing masks
import os
from PIL import Image
import pandas as pd

# Define the paths
csv_path = '/content/missing_images.csv'
images_root_path = '/content/Shenzhen/images/images'
masks_root_path = '/content/masks/AllEnh5Dill10Aug4/Shenzen'
output_path = '/content/Shenzhen_segmented/Available_Masks'

# Create the output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Load the CSV file
df = pd.read_csv(csv_path)

# Get the list of image/mask names from the study_id column
image_names = df['study_id'].tolist()

# Process each image and its corresponding mask
for img_name in image_names:
    # Construct the full image and mask paths
    image_path = os.path.join(images_root_path, img_name)
    mask_path = os.path.join(masks_root_path, img_name)

    # Check if both the image and mask exist
    if os.path.exists(image_path) and os.path.exists(mask_path):
        # Load image and mask
        img = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        # Resize the image and mask to (224, 224)
        img = img.resize((224, 224))
        mask = mask.resize((224, 224))

        # Apply mask to image
        segmented_img = Image.composite(img, Image.new('RGB', img.size, (0, 0, 0)), mask)

        # Save the segmented image
        segmented_img.save(os.path.join(output_path, img_name))
    else:
        print(f"Image or mask for {img_name} not found. Skipping this item.")

print("Segmentation complete. Segmented images saved to:", output_path)


Segmentation complete. Segmented images saved to: /content/Shenzhen_segmented/Available_Masks


In [29]:
#Shenzhen Custom Model Segmentation
from PIL import Image
import os

# Define the paths
images_path = '/content/Shenzhen/images/images'
masks_path = '/content/masks/AllEnh5Dill10Aug4/Shenzen'
output_path = '/content/Shenzhen_segmented/Custom_masks'

# Create the output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# List all images
images = sorted(os.listdir(images_path))

# Process each image and its corresponding mask
for img_name in images:
    # Construct mask name assuming it follows the same naming convention as images
    mask_name = img_name  # Adjust this if the naming convention is different

    mask_path = os.path.join(masks_path, mask_name)

    # Check if the corresponding mask exists
    if os.path.exists(mask_path):
        # Load image and mask
        img = Image.open(os.path.join(images_path, img_name)).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        # Resize the image and mask to (224, 224)
        img = img.resize((224, 224))
        mask = mask.resize((224, 224))

        # Apply mask to image
        segmented_img = Image.composite(img, Image.new('RGB', img.size, (0, 0, 0)), mask)

        # Save the segmented image
        segmented_img.save(os.path.join(output_path, img_name))
    else:
        print(f"Mask for {img_name} not found. Skipping this image.")

print("Segmentation complete. Segmented images saved to:", output_path)


Segmentation complete. Segmented images saved to: /content/Shenzhen_segmented/Custom_masks


In [31]:
#4200 image Normal
from PIL import Image
import os

# Define the paths
images_path = '/content/4200images/TB_Chest_Radiography_Database/Normal'
masks_path = '/content/masks/AllEnh5Dill10Aug4/4200/Normal'
output_path = '/content/4200 Segmented/Normal'

# Create the output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# List all images
images = sorted(os.listdir(images_path))

# Process each image and its corresponding mask
for img_name in images:
    # Construct mask name assuming it follows the same naming convention as images
    mask_name = img_name  # Adjust this if the naming convention is different

    mask_path = os.path.join(masks_path, mask_name)

    # Check if the corresponding mask exists
    if os.path.exists(mask_path):
        # Load image and mask
        img = Image.open(os.path.join(images_path, img_name)).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        # Resize the image and mask to (224, 224)
        img = img.resize((224, 224))
        mask = mask.resize((224, 224))

        # Apply mask to image
        segmented_img = Image.composite(img, Image.new('RGB', img.size, (0, 0, 0)), mask)

        # Save the segmented image
        segmented_img.save(os.path.join(output_path, img_name))
    else:
        print(f"Mask for {img_name} not found. Skipping this image.")

print("Segmentation complete. Segmented images saved to:", output_path)


Segmentation complete. Segmented images saved to: /content/4200 Segmented/Normal


In [32]:
#4200 image TB
from PIL import Image
import os

# Define the paths
images_path = '/content/4200images/TB_Chest_Radiography_Database/Tuberculosis'
masks_path = '/content/masks/AllEnh5Dill10Aug4/4200/Tuberculosis'
output_path = '/content/4200 Segmented/Tuberculosis'

# Create the output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# List all images
images = sorted(os.listdir(images_path))

# Process each image and its corresponding mask
for img_name in images:
    # Construct mask name assuming it follows the same naming convention as images
    mask_name = img_name  # Adjust this if the naming convention is different

    mask_path = os.path.join(masks_path, mask_name)

    # Check if the corresponding mask exists
    if os.path.exists(mask_path):
        # Load image and mask
        img = Image.open(os.path.join(images_path, img_name)).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        # Resize the image and mask to (224, 224)
        img = img.resize((224, 224))
        mask = mask.resize((224, 224))

        # Apply mask to image
        segmented_img = Image.composite(img, Image.new('RGB', img.size, (0, 0, 0)), mask)

        # Save the segmented image
        segmented_img.save(os.path.join(output_path, img_name))
    else:
        print(f"Mask for {img_name} not found. Skipping this image.")

print("Segmentation complete. Segmented images saved to:", output_path)

Segmentation complete. Segmented images saved to: /content/4200 Segmented/Tuberculosis


## Division

In [34]:
#Montgomery
import pandas as pd

# Path to the CSV file
csv_path = "/content/montgomery/montgomery_metadata.csv"

# Read the CSV file
df = pd.read_csv(csv_path)

# Drop the "age" and "gender" columns
df = df.drop(columns=["age", "gender"], errors='ignore')

# Update the "findings" column
df["findings"] = df["findings"].apply(lambda x: "normal" if x == "normal" else "tuberculosis")

# Save the modified DataFrame back to a CSV file
df.to_csv("/content/montgomery_metadata_modified.csv", index=False)

print("CSV file modified successfully!")


CSV file modified successfully!


In [35]:
# Available masks
import pandas as pd
import os
import shutil

# Paths
csv_path = "/content/montgomery_metadata_modified.csv"
images_path = "/content/Montgomery_segmented/Available_masks"
normal_path = "/content/Montgomery_segmented/Montgomery_Available_masks/Normal"
tuberculosis_path = "/content/Montgomery_segmented/Montgomery_Available_masks/Tuberculosis"

# Create output directories if they don't exist
os.makedirs(normal_path, exist_ok=True)
os.makedirs(tuberculosis_path, exist_ok=True)

# Read the modified CSV file
df = pd.read_csv(csv_path)

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    image_filename = f"{row['study_id']}"  # Assuming the column is named 'filename'
    label = row["findings"]

    # Construct source and destination paths
    source_file = os.path.join(images_path, image_filename)

    if label == "normal":
        shutil.copy(source_file, normal_path)
    else:
        shutil.copy(source_file, tuberculosis_path)

print("Images have been organized into 'Normal' and 'Tuberculosis' folders!")


Images have been organized into 'Normal' and 'Tuberculosis' folders!


In [36]:
#Custom Masks
import pandas as pd
import os
import shutil

# Paths
csv_path = "/content/montgomery_metadata_modified.csv"
images_path = "/content/Montgomery_segmented/Custom_masks"
normal_path = "/content/Montgomery_segmented/Montgomery_Custom_Masks/Normal"
tuberculosis_path = "/content/Montgomery_segmented/Montgomery_Custom_Masks/Tuberculosis"

# Create output directories if they don't exist
os.makedirs(normal_path, exist_ok=True)
os.makedirs(tuberculosis_path, exist_ok=True)

# Read the modified CSV file
df = pd.read_csv(csv_path)

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    image_filename = f"{row['study_id']}"  # Assuming the column is named 'filename'
    label = row["findings"]

    # Construct source and destination paths
    source_file = os.path.join(images_path, image_filename)

    if label == "normal":
        shutil.copy(source_file, normal_path)
    else:
        shutil.copy(source_file, tuberculosis_path)

print("Images have been organized into 'Normal' and 'Tuberculosis' folders!")

Images have been organized into 'Normal' and 'Tuberculosis' folders!


In [38]:
#Shenzhen
import pandas as pd
import os
import shutil

# Paths
csv_path = "/content/Shenzhen/shenzhen_metadata.csv"
images_path = "/content/Shenzhen_segmented/Available_Masks"
normal_path = "/content/Shenzhen_segmented/Shenzhen_Available_Masks/Normal"
tuberculosis_path = "/content/Shenzhen_segmented/Shenzhen_Available_Masks/Tuberculosis"

# Create output directories if they don't exist
os.makedirs(normal_path, exist_ok=True)
os.makedirs(tuberculosis_path, exist_ok=True)

# Read the modified CSV file
df = pd.read_csv(csv_path)

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    image_filename = f"{row['study_id']}"  # Assuming the column is named 'filename'
    label = row["findings"]

    # Construct source and destination paths
    source_file = os.path.join(images_path, image_filename)

    if label == "normal":
        shutil.copy(source_file, normal_path)
    else:
        shutil.copy(source_file, tuberculosis_path)

print("Images have been organized into 'Normal' and 'Tuberculosis' folders!")


Images have been organized into 'Normal' and 'Tuberculosis' folders!


In [40]:
#Shenzhen Custom Masks
import pandas as pd
import os
import shutil

# Paths
csv_path = "/content/Shenzhen/shenzhen_metadata.csv"
images_path = "/content/Shenzhen_segmented/Custom_masks"
normal_path = "/content/Shenzhen_segmented/Shenzhen_Custom_Masks/Normal"
tuberculosis_path = "/content/Shenzhen_segmented/Shenzhen_Custom_Masks/Tuberculosis"

# Create output directories if they don't exist
os.makedirs(normal_path, exist_ok=True)
os.makedirs(tuberculosis_path, exist_ok=True)

# Read the modified CSV file
df = pd.read_csv(csv_path)

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    image_filename = f"{row['study_id']}"  # Assuming the column is named 'filename'
    label = row["findings"]

    # Construct source and destination paths
    source_file = os.path.join(images_path, image_filename)

    if label == "normal":
        shutil.copy(source_file, normal_path)
    else:
        shutil.copy(source_file, tuberculosis_path)

print("Images have been organized into 'Normal' and 'Tuberculosis' folders!")

Images have been organized into 'Normal' and 'Tuberculosis' folders!


In [49]:
#Counting
import os

# Path to the main folder
main_path = "/content/All images combined"

# Iterate through each subfolder and count images
for subfolder in os.listdir(main_path):
    subfolder_path = os.path.join(main_path, subfolder)
    if os.path.isdir(subfolder_path):
        for sub_subfolder in os.listdir(subfolder_path):
            sub_subfolder_path = os.path.join(subfolder_path, sub_subfolder)
            if os.path.isdir(sub_subfolder_path):
                image_count = len([f for f in os.listdir(sub_subfolder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
                print(f"{subfolder}/{sub_subfolder}: {image_count} images")


Montgomery_Custom_Masks/Tuberculosis: 58 images
Montgomery_Custom_Masks/Normal: 80 images
Merged_Available/Tuberculosis: 1094 images
Merged_Available/Normal: 3906 images
4200 Segmented/Tuberculosis: 700 images
4200 Segmented/Normal: 3500 images
Shenzhen_Custom_Masks/Tuberculosis: 336 images
Shenzhen_Custom_Masks/Normal: 326 images
Montgomery_Available_masks/Tuberculosis: 58 images
Montgomery_Available_masks/Normal: 80 images
S+M_Custom/Tuberculosis: 394 images
S+M_Custom/Normal: 406 images
S+M_Available/Tuberculosis: 394 images
S+M_Available/Normal: 406 images
Shenzhen_Available_Masks/Tuberculosis: 336 images
Shenzhen_Available_Masks/Normal: 326 images
Merged_Custom/Tuberculosis: 1094 images
Merged_Custom/Normal: 3906 images


In [48]:
import os
import cv2

# Path to the main folder
main_path = "/content/All images combined"

# Initialize a counter for resized images
resized_count = 0

# Iterate through each subfolder
for subfolder in os.listdir(main_path):
    subfolder_path = os.path.join(main_path, subfolder)
    if os.path.isdir(subfolder_path):
        for sub_subfolder in os.listdir(subfolder_path):
            sub_subfolder_path = os.path.join(subfolder_path, sub_subfolder)
            if os.path.isdir(sub_subfolder_path):
                # Iterate through each image in the sub-subfolder
                for image_file in os.listdir(sub_subfolder_path):
                    image_path = os.path.join(sub_subfolder_path, image_file)
                    if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        # Read and resize the image
                        image = cv2.imread(image_path)
                        if image is not None:
                            resized_image = cv2.resize(image, (224, 224))

                            # Save the resized image, replacing the original
                            cv2.imwrite(image_path, resized_image)

                            # Increment the counter
                            resized_count += 1

# Display the total number of images resized
print(f"All images have been resized to (224, 224, 3)! Total resized images: {resized_count}")


All images have been resized to (224, 224, 3)! Total resized images: 17400


In [44]:
import os
import shutil

# Define the source directories
source_dirs = [
    "/content/All images combined/4200 Segmented",
    "/content/All images combined/Montgomery_Available_masks",
    "/content/All images combined/Shenzhen_Available_Masks"
]

# Define the destination directory
dest_dir = "/content/All images combined/Merged_Available"

# Ensure the destination directories exist
os.makedirs(os.path.join(dest_dir, "Normal"), exist_ok=True)
os.makedirs(os.path.join(dest_dir, "Tuberculosis"), exist_ok=True)

# Function to copy images from source to destination
def copy_images(src, dest):
    for root, dirs, files in os.walk(src):
        for file in files:
            if file.endswith(('png', 'jpg', 'jpeg', 'bmp')):  # Add more extensions if needed
                src_file = os.path.join(root, file)
                dest_file = os.path.join(dest, file)
                shutil.copy(src_file, dest_file)

# Copy images from each source directory to the corresponding destination subdirectory
for source_dir in source_dirs:
    normal_src = os.path.join(source_dir, "Normal")
    tuberculosis_src = os.path.join(source_dir, "Tuberculosis")

    if os.path.exists(normal_src):
        copy_images(normal_src, os.path.join(dest_dir, "Normal"))

    if os.path.exists(tuberculosis_src):
        copy_images(tuberculosis_src, os.path.join(dest_dir, "Tuberculosis"))

print("Images combined successfully.")


Images combined successfully.


In [45]:
import os
import shutil

# Define the source directories
source_dirs = [
    "/content/All images combined/4200 Segmented",
    "/content/All images combined/Montgomery_Custom_Masks",
    "/content/All images combined/Shenzhen_Custom_Masks"
]

# Define the destination directory
dest_dir = "/content/All images combined/Merged_Custom"

# Ensure the destination directories exist
os.makedirs(os.path.join(dest_dir, "Normal"), exist_ok=True)
os.makedirs(os.path.join(dest_dir, "Tuberculosis"), exist_ok=True)

# Function to copy images from source to destination
def copy_images(src, dest):
    for root, dirs, files in os.walk(src):
        for file in files:
            if file.endswith(('png', 'jpg', 'jpeg', 'bmp')):  # Add more extensions if needed
                src_file = os.path.join(root, file)
                dest_file = os.path.join(dest, file)
                shutil.copy(src_file, dest_file)

# Copy images from each source directory to the corresponding destination subdirectory
for source_dir in source_dirs:
    normal_src = os.path.join(source_dir, "Normal")
    tuberculosis_src = os.path.join(source_dir, "Tuberculosis")

    if os.path.exists(normal_src):
        copy_images(normal_src, os.path.join(dest_dir, "Normal"))

    if os.path.exists(tuberculosis_src):
        copy_images(tuberculosis_src, os.path.join(dest_dir, "Tuberculosis"))

print("Images combined successfully.")

Images combined successfully.


In [47]:
import os
import shutil

# Define the source directories
source_dirs = [
    "/content/All images combined/Montgomery_Available_masks",
    "/content/All images combined/Shenzhen_Available_Masks"
]

# Define the destination directory
dest_dir = "/content/All images combined/S+M_Available"

# Ensure the destination directories exist
os.makedirs(os.path.join(dest_dir, "Normal"), exist_ok=True)
os.makedirs(os.path.join(dest_dir, "Tuberculosis"), exist_ok=True)

# Function to copy images from source to destination
def copy_images(src, dest):
    for root, dirs, files in os.walk(src):
        for file in files:
            if file.endswith(('png', 'jpg', 'jpeg', 'bmp')):  # Add more extensions if needed
                src_file = os.path.join(root, file)
                dest_file = os.path.join(dest, file)
                shutil.copy(src_file, dest_file)

# Copy images from each source directory to the corresponding destination subdirectory
for source_dir in source_dirs:
    normal_src = os.path.join(source_dir, "Normal")
    tuberculosis_src = os.path.join(source_dir, "Tuberculosis")

    if os.path.exists(normal_src):
        copy_images(normal_src, os.path.join(dest_dir, "Normal"))

    if os.path.exists(tuberculosis_src):
        copy_images(tuberculosis_src, os.path.join(dest_dir, "Tuberculosis"))

print("Images combined successfully.")

Images combined successfully.


In [46]:
import os
import shutil

# Define the source directories
source_dirs = [
    "/content/All images combined/Montgomery_Custom_Masks",
    "/content/All images combined/Shenzhen_Custom_Masks"
]

# Define the destination directory
dest_dir = "/content/All images combined/S+M_Custom"

# Ensure the destination directories exist
os.makedirs(os.path.join(dest_dir, "Normal"), exist_ok=True)
os.makedirs(os.path.join(dest_dir, "Tuberculosis"), exist_ok=True)

# Function to copy images from source to destination
def copy_images(src, dest):
    for root, dirs, files in os.walk(src):
        for file in files:
            if file.endswith(('png', 'jpg', 'jpeg', 'bmp')):  # Add more extensions if needed
                src_file = os.path.join(root, file)
                dest_file = os.path.join(dest, file)
                shutil.copy(src_file, dest_file)

# Copy images from each source directory to the corresponding destination subdirectory
for source_dir in source_dirs:
    normal_src = os.path.join(source_dir, "Normal")
    tuberculosis_src = os.path.join(source_dir, "Tuberculosis")

    if os.path.exists(normal_src):
        copy_images(normal_src, os.path.join(dest_dir, "Normal"))

    if os.path.exists(tuberculosis_src):
        copy_images(tuberculosis_src, os.path.join(dest_dir, "Tuberculosis"))

print("Images combined successfully.")

Images combined successfully.


In [50]:
#Zipping
import shutil

# Path to the folder to zip
folder_path = "/content/All images combined"
zip_path = "/content/All images combined.zip"

# Create a zip file
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)

print("Folder has been zipped successfully!")


Folder has been zipped successfully!


**Preprocessing** (histogram & Clahe)

In [ ]:
import cv2
import os

# Define the folder path containing the images
folder_path = '/content/4200'

# Recursively traverse the directory tree
for root, _, files in os.walk(folder_path):
    for file in files:
        image_path = os.path.join(root, file)

        # Only process files with valid image extensions
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Check if the image was successfully loaded
            if image is None:
                print(f"Image at path '{image_path}' not found.")
                continue

            # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            clahe_image = clahe.apply(image)

            # Apply histogram equalization on the original image (not on CLAHE result)
            hist_eq_image = cv2.equalizeHist(image)

            # Apply CLAHE and then histogram equalization
            hist_eq_then_clahe_image = clahe.apply(hist_eq_image)

            # Save the modified image, replacing the original
            cv2.imwrite(image_path, hist_eq_then_clahe_image)

            print(f"Processed and replaced image at path '{image_path}'")


Processed and replaced image at path '/content/4200/tb/Tuberculosis-166.png'
Processed and replaced image at path '/content/4200/tb/Tuberculosis-642.png'
Processed and replaced image at path '/content/4200/tb/Tuberculosis-395.png'
Processed and replaced image at path '/content/4200/tb/Tuberculosis-104.png'
Processed and replaced image at path '/content/4200/tb/Tuberculosis-487.png'
Processed and replaced image at path '/content/4200/tb/Tuberculosis-499.png'
Processed and replaced image at path '/content/4200/tb/Tuberculosis-246.png'
Processed and replaced image at path '/content/4200/tb/Tuberculosis-657.png'
Processed and replaced image at path '/content/4200/tb/Tuberculosis-157.png'
Processed and replaced image at path '/content/4200/tb/Tuberculosis-304.png'
Processed and replaced image at path '/content/4200/tb/Tuberculosis-662.png'
Processed and replaced image at path '/content/4200/tb/Tuberculosis-597.png'
Processed and replaced image at path '/content/4200/tb/Tuberculosis-436.png'